In [3]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [4]:
rawdata = pd.read_csv('data/milk.csv')
data = rawdata.copy()

data = rawdata.copy()

#lag auctions within 1 vendor
data = data.sort_values(['VENDOR','YEAR','MONTH','DAY','SYSTEM'])
bids = data.groupby(['VENDOR','YEAR','MONTH','DAY','SYSTEM'], as_index=False).mean()
bids = bids[['VENDOR','YEAR','MONTH','DAY','SYSTEM']]
sys_lag = bids.groupby(['VENDOR']).shift(1)
bids['VENDOR_LAG'] = sys_lag['SYSTEM']
data = pd.merge(data, bids, how='left', 
                 on=['VENDOR','YEAR','MONTH','DAY','SYSTEM'], suffixes=('', '_LAG') ) 

#lag auctions
data =data.sort_values(['YEAR','MONTH','DAY','SYSTEM'])
aucts =  data.groupby(['YEAR','MONTH','DAY','SYSTEM'], as_index=False).mean()
aucts = aucts[['YEAR','MONTH','DAY','SYSTEM']]
auct_lag = aucts.shift(1)
aucts['AUCT_LAG'] = auct_lag['SYSTEM']
data = pd.merge(data, aucts, how='left', 
                 on=['YEAR','MONTH','DAY','SYSTEM'], suffixes=('', '_LAG') ) 

prev_auct = 1.*(data['AUCT_LAG'] == data['VENDOR_LAG'])
data['PAST_AUCT'] = prev_auct

In [5]:
#general house keeping
data = data[ (data['YEAR']>=1980)]
data = data[(data['MONTH'] >= 4) & (data['MONTH'] <= 9) & (data['DAY'] !=0) ] #need data with time index
data = data[(~np.isnan(data['SCORE']) ) & (data['QSCORE']!=0 )] #need data with QWW and WW


data['COOLER'] = data['COOLER'].fillna(0)
data['ESC'] = data['ESC'].fillna(0)
data['ONEBID'] = 1.*(data['NUM'] == 1)

#various keys
milk = ['SCORE']
auct_key = ['YEAR','MONTH','DAY','SYSTEM','FMOZONE']
cts = ['FMO','GAS','POPUL','QSCORE']
dummies = ['COOLER','ESC', 'NUM'] #delete num?


#baseline stuff/logs
lmilk = ['L'+x for x in milk]
lcts = ['L'+x for x in cts]
data[lcts] = np.log(data[cts])
data[lmilk] = np.log(data[milk])

#set up lags
lags = 10
lagkeys = [l+str(i) for l in ['LSCORE_min','LSCORE_max'] for i in range(1,1+lags)]
aucts = data.groupby(auct_key, as_index=False)[milk].mean()[auct_key]

#note data is already sorted by date
data = data.sort_values(['YEAR','MONTH','DAY'])
min_lag = data.groupby(auct_key, as_index=False).min()
for t in range(1,1+lags):
    min_lagt = min_lag.shift(t)[lmilk]
    min_lagt = pd.concat((aucts, min_lagt), axis=1)
    data = pd.merge(data, min_lagt, how='left', on=auct_key, suffixes=('', '_min%s'%(t)) ) 
    
max_lag = data.groupby(auct_key, as_index=False).max()
for t in range(1,1+lags):
    max_lagt = max_lag.shift(t)[lmilk]
    max_lagt = pd.concat((aucts, max_lagt), axis=1)
    data = pd.merge(data, max_lagt, how='left', on=auct_key, suffixes=('', '_max%s'%(t)) )    

    
#set up lags but prev year 
yearlags = 1
year_lagkeys = [l+str(i) for l in ['LSCORE_miny','LSCORE_maxy'] for i in range(1,1+yearlags)]


#note data is sorted by year now
data = data.sort_values(['SYSTEM','YEAR','MONTH','DAY'])

year_min_lag = data.groupby(auct_key, as_index=False).min().sort_values(['SYSTEM','YEAR','MONTH','DAY'])
for t in range(1,1+lags):
    min_lagt = year_min_lag.groupby(['SYSTEM']).shift(t)[lmilk]
    min_lagt = pd.concat((aucts, min_lagt), axis=1)
    data = pd.merge(data, min_lagt, how='left', on=auct_key, suffixes=('', '_miny%s'%(t)) )

year_max_lag = data.groupby(auct_key, as_index=False).max().sort_values(['SYSTEM','YEAR','MONTH','DAY'])
for t in range(1,1+lags):
    max_lagt = year_max_lag.groupby(['SYSTEM']).shift(t)[lmilk]
    max_lagt = pd.concat((aucts, max_lagt), axis=1)
    data = pd.merge(data, max_lagt, how='left', on=auct_key, suffixes=('', '_maxy%s'%(t)) ) 
    

#pre processing to help fmozones
fe = ['FMOZONE']
data.loc[(data['FMOZONE'] =='1A') , 'FMOZONE'] = '1'

fekeys = []
for effect in fe:
    fes = pd.get_dummies(data[effect], drop_first=True)
    fekeys = fekeys+ list(fes.columns)
    data = pd.concat((data, fes), axis=1)

    
bid_key = auct_key + ['VENDOR'] + ['COUNTY']
covariates = lcts + dummies + fekeys
hist = ['INC'] + lagkeys 

#setup interaction term for punishment
data['min*past'] = data['LSCORE_min1']*data['PAST_AUCT']

In [6]:
#only drop data 5 periods back
maxlag = 4
limitedlag = [l+str(i) for l in ['LSCORE_min','LSCORE_max'] for i in range(1,1+maxlag)]


maxlagy = 2
limitedlagy = [l+str(i) for l in ['LSCORE_miny','LSCORE_maxy'] for i in range(1,1+maxlagy)]


limitedhist = ['PAST_AUCT','min*past'] + limitedlag + limitedlagy 


reg1 = data.copy()[bid_key + lmilk + covariates + limitedhist + ['INC','WIN']]
reg1 = reg1.dropna()
reg1.to_csv('data/clean_milk1.csv')

In [13]:
#baseline
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates] ) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     56.80
Date:                Sun, 21 Jul 2019   Prob (F-statistic):          2.60e-115
Time:                        14:34:17   Log-Likelihood:                 3183.8
No. Observations:                3153   AIC:                            -6344.
Df Residuals:                    3141   BIC:                            -6271.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2682      0.085    -26.614      0.000      -2.435      -2.101
LFMO           0.2227      0.030      7.305      0.000       0.163       0.283
LGAS           0.0090      0.005      1.648      0.099      -0.002       0.020
LPOPUL         0.0159      0.004      3.853      0.000       0.008       0.024
LQSCORE       -0.0213      0.004     -4.938      0.000      -0.030      -0.013
COOLER         0.0177      0.004      4.984      0.000       0.011       0.025
ESC           -0.0294      0.003     -8.895      0.000      -0.036      -0.023
NUM            0.0078      0.002      4.709      0.000       0.005       0.011
3             -0.0697      0.005    -13.592      0.000      -0.080      -0.060
6             -0.0413      0.015     -2.686      0.007      -0.072      -0.011
7             -0.1068      0.017     -6.417      0.000      -0.139      -0.074
9             -0.0520      0.005    -11.076      0.000      -0.061      -0.043
==============================================================================
Omnibus:                      268.816   Durbin-Watson:                   1.198
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1444.596
Skew:                           0.201   Prob(JB):                         0.00
Kurtosis:                       6.291   Cond. No.                         946.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
#incumbency
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates+ ['INC']] ) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     58.78
Date:                Sun, 21 Jul 2019   Prob (F-statistic):          1.11e-128
Time:                        14:34:17   Log-Likelihood:                 3217.2
No. Observations:                3153   AIC:                            -6408.
Df Residuals:                    3140   BIC:                            -6330.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2384      0.084    -26.515      0.000      -2.404      -2.073
LFMO           0.2168      0.030      7.181      0.000       0.158       0.276
LGAS           0.0094      0.005      1.749      0.080      -0.001       0.020
LPOPUL         0.0154      0.004      3.777      0.000       0.007       0.023
LQSCORE       -0.0208      0.004     -4.872      0.000      -0.029      -0.012
COOLER         0.0174      0.004      4.928      0.000       0.010       0.024
ESC           -0.0304      0.003     -9.284      0.000      -0.037      -0.024
NUM            0.0053      0.002      3.172      0.002       0.002       0.009
3             -0.0708      0.005    -13.947      0.000      -0.081      -0.061
6             -0.0409      0.015     -2.689      0.007      -0.071      -0.011
7             -0.1064      0.016     -6.457      0.000      -0.139      -0.074
9             -0.0540      0.005    -11.600      0.000      -0.063      -0.045
INC           -0.0306      0.004     -8.207      0.000      -0.038      -0.023
==============================================================================
Omnibus:                      285.446   Durbin-Watson:                   1.101
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1685.007
Skew:                           0.193   Prob(JB):                         0.00
Kurtosis:                       6.560   Cond. No.                         947.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
#2 lines incumbency

In [16]:
#previous prices 1 day
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates + ['INC'] + limitedlag]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     44.62
Date:                Sun, 21 Jul 2019   Prob (F-statistic):          6.70e-154
Time:                        14:34:18   Log-Likelihood:                 3293.0
No. Observations:                3153   AIC:                            -6544.
Df Residuals:                    3132   BIC:                            -6417.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.4170      0.117    -12.163      0.000      -1.645      -1.189
LFMO            0.1420      0.030      4.663      0.000       0.082       0.202
LGAS            0.0113      0.005      2.141      0.032       0.001       0.022
LPOPUL          0.0089      0.004      2.217      0.027       0.001       0.017
LQSCORE        -0.0169      0.004     -4.019      0.000      -0.025      -0.009
COOLER          0.0163      0.003      4.725      0.000       0.010       0.023
ESC            -0.0286      0.003     -8.911      0.000      -0.035      -0.022
NUM             0.0035      0.002      2.118      0.034       0.000       0.007
3              -0.0727      0.005    -14.599      0.000      -0.082      -0.063
6              -0.0440      0.015     -2.930      0.003      -0.073      -0.015
7              -0.1083      0.016     -6.681      0.000      -0.140      -0.077
9              -0.0595      0.005    -12.959      0.000      -0.069      -0.051
INC            -0.0327      0.004     -8.954      0.000      -0.040      -0.026
LSCORE_min1    -0.0511      0.019     -2.631      0.009      -0.089      -0.013
LSCORE_min2     0.0271      0.020      1.356      0.175      -0.012       0.066
LSCORE_min3     0.0704      0.020      3.527      0.000       0.031       0.110
LSCORE_min4     0.0040      0.020      0.199      0.842      -0.035       0.043
LSCORE_max1     0.1533      0.021      7.393      0.000       0.113       0.194
LSCORE_max2     0.0531      0.021      2.525      0.012       0.012       0.094
LSCORE_max3     0.0588      0.020      2.878      0.004       0.019       0.099
LSCORE_max4     0.0366      0.020      1.871      0.061      -0.002       0.075
==============================================================================
Omnibus:                      343.495   Durbin-Watson:                   1.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2090.461
Skew:                           0.325   Prob(JB):                         0.00
Kurtosis:                       6.936   Cond. No.                     1.36e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
#prev prices with prev auction
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates+ ['PAST_AUCT','min*past'] + ['INC']+ limitedlag ]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     41.12
Date:                Sun, 21 Jul 2019   Prob (F-statistic):          2.32e-154
Time:                        14:34:19   Log-Likelihood:                 3298.0
No. Observations:                3153   AIC:                            -6550.
Df Residuals:                    3130   BIC:                            -6411.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.4706      0.119    -12.378      0.000      -1.704      -1.238
LFMO            0.1373      0.030      4.508      0.000       0.078       0.197
LGAS            0.0113      0.005      2.135      0.033       0.001       0.022
LPOPUL          0.0088      0.004      2.189      0.029       0.001       0.017
LQSCORE        -0.0169      0.004     -4.017      0.000      -0.025      -0.009
COOLER          0.0158      0.003      4.561      0.000       0.009       0.023
ESC            -0.0286      0.003     -8.915      0.000      -0.035      -0.022
NUM             0.0034      0.002      2.060      0.039       0.000       0.007
3              -0.0732      0.005    -14.676      0.000      -0.083      -0.063
6              -0.0429      0.015     -2.861      0.004      -0.072      -0.014
7              -0.1082      0.016     -6.683      0.000      -0.140      -0.076
9              -0.0591      0.005    -12.879      0.000      -0.068      -0.050
PAST_AUCT       0.1596      0.062      2.571      0.010       0.038       0.281
min*past        0.0900      0.034      2.660      0.008       0.024       0.156
INC            -0.0330      0.004     -9.035      0.000      -0.040      -0.026
LSCORE_min1    -0.0978      0.025     -3.891      0.000      -0.147      -0.049
LSCORE_min2     0.0268      0.020      1.340      0.180      -0.012       0.066
LSCORE_min3     0.0704      0.020      3.532      0.000       0.031       0.110
LSCORE_min4     0.0023      0.020      0.116      0.907      -0.037       0.042
LSCORE_max1     0.1636      0.021      7.773      0.000       0.122       0.205
LSCORE_max2     0.0533      0.021      2.540      0.011       0.012       0.095
LSCORE_max3     0.0591      0.020      2.896      0.004       0.019       0.099
LSCORE_max4     0.0366      0.020      1.870      0.062      -0.002       0.075
==============================================================================
Omnibus:                      336.012   Durbin-Watson:                   1.161
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2032.118
Skew:                           0.311   Prob(JB):                         0.00
Kurtosis:                       6.883   Cond. No.                     1.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
#previous prices prev year
sm.OLS(reg1['LSCORE'], sm.add_constant(reg1[covariates+ ['INC'] + limitedhist]) ).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 LSCORE   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     97.37
Date:                Sun, 21 Jul 2019   Prob (F-statistic):               0.00
Time:                        14:34:19   Log-Likelihood:                 3833.0
No. Observations:                3153   AIC:                            -7612.
Df Residuals:                    3126   BIC:                            -7448.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.8904      0.104     -8.581      0.000      -1.094      -0.687
LFMO             0.3126      0.026     11.852      0.000       0.261       0.364
LGAS            -0.0166      0.005     -3.651      0.000      -0.026      -0.008
LPOPUL           0.0076      0.003      2.247      0.025       0.001       0.014
LQSCORE         -0.0133      0.004     -3.752      0.000      -0.020      -0.006
COOLER           0.0001      0.003      0.035      0.972      -0.006       0.006
ESC             -0.0203      0.003     -7.418      0.000      -0.026      -0.015
NUM              0.0038      0.001      2.700      0.007       0.001       0.007
3               -0.0308      0.004     -6.930      0.000      -0.039      -0.022
6               -0.0298      0.013     -2.350      0.019      -0.055      -0.005
7               -0.0107      0.014     -0.763      0.446      -0.038       0.017
9               -0.0123      0.004     -2.988      0.003      -0.020      -0.004
INC             -0.0408      0.003    -13.197      0.000      -0.047      -0.035
PAST_AUCT        0.1094      0.052      2.085      0.037       0.007       0.212
min*past         0.0634      0.029      2.219      0.027       0.007       0.119
LSCORE_min1     -0.0827      0.021     -3.892      0.000      -0.124      -0.041
LSCORE_min2      0.0420      0.017      2.476      0.013       0.009       0.075
LSCORE_min3      0.0169      0.017      1.001      0.317      -0.016       0.050
LSCORE_min4      0.0211      0.017      1.249      0.212      -0.012       0.054
LSCORE_max1      0.1079      0.018      6.032      0.000       0.073       0.143
LSCORE_max2      0.0243      0.018      1.366      0.172      -0.011       0.059
LSCORE_max3      0.0369      0.017      2.134      0.033       0.003       0.071
LSCORE_max4      0.0453      0.017      2.743      0.006       0.013       0.078
LSCORE_miny1     0.4871      0.024     20.628      0.000       0.441       0.533
LSCORE_miny2     0.0290      0.024      1.207      0.228      -0.018       0.076
LSCORE_maxy1     0.1258      0.020      6.228      0.000       0.086       0.165
LSCORE_maxy2     0.0172      0.020      0.878      0.380      -0.021       0.056
==============================================================================
Omnibus:                      334.961   Durbin-Watson:                   1.541
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2242.443
Skew:                           0.259   Prob(JB):                         0.00
Kurtosis:                       7.099   Cond. No.                     1.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+03. This might indicate that there 